In [169]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

import numpy as np

import pandas as pd
import json
import janitor

from easyeditor.custom import * # gets my custom functions

from ast import literal_eval

/scratch/dmpowell/.cache/huggingface


In [170]:
def load_result(filename):
    x = pd.read_csv(filename, converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
    return(x)

res = load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv")

In [193]:
# define reporting function
def report_results(df):
    
    # out = (
    #     df
    #     .assign(
    #         chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
    #         chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
    #     )
    #     .filter(['edit_type','entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
    #     .pivot_longer(
    #         index = ['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
    #         names_to = ('var', 'query_type'),
    #         names_sep = '_'
    #     )
    #     # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
    #     .assign(test_group = lambda x: np.select(
    #         [(x.edit_type == "category property") & (x.subj == x.entity), x.edit_type == "category property", x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
    #         ["CP: category property", "CP: token property", "CM: category membership (exact)", "CM: category membership (paraphrase)", "CM: token property"]
    #         ))
    #     .groupby(['test_group', 'var'])
    #     .agg(
    #         prop = ('value', 'mean')
    #         )
    #     .reset_index()
    #     .pivot(index = ['test_group'], columns = ['var'], values = 'prop')

    # )
     
    out2 = (
        df    
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['edit_type','entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [(x.edit_type == "category property") & (x.subj == x.entity), (x.edit_type == "category property") & (x.subj != x.entity), x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["CP: category property", "CP: token property", "CM: category membership (exact)", "CM: category membership (paraphrase)", "CM: token property"]
            ))
        .groupby(['test_group', 'query_type', "token_type", 'var'])
        .agg(
            prop = ('value', 'mean')
            )
        .reset_index()
        .pivot(index = ['test_group','query_type', "token_type"], columns = ['var'], values = 'prop')

    )

    return pd.concat([out2])


  

In [223]:
res_ft = load_result("results/csv/meta-llama-Llama-2-7b-hf-FT.csv")

# (
#     res_ft
#     .loc[lambda x: x.edit_type == "category property"]
#     .filter(["entity", "query_fwd", "subj", "answer_fwd", "fwd_choices", "fwd_predicted"])
#     .loc[lambda x: x.fwd_predicted == 0]
# )

# res_ft.columns


## overview

Two edit types

1. Category-membership: Editing the category to which an exemplar belongs (e.g. "a Labrador is kind of a cow")
2. Category-property: Editing a property of a category (e.g. "a sound a dog makes is moo")

2 test types

1. Edit confirmation
2. Generalization:
    1. For CM: to properties of the edited exemplar (typical and atypical)
    2. for CP: to exemplars of the edited category (typical and atypical)

2 query types (forward and reverse)

Currently, the Category-property edits aren't really working (they are slightly above chance but generally not working). I'm not sure why this is! for friday feb 9 just need to ignore that and press on to the other categories of edits.

In [224]:
report_results(res_ft)

/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


var                                                      chance   correct
test_group                      query_type token_type                    
CM: category membership (exact) fwd        rare        0.146495  0.944444
                                           typical     0.138492  0.933333
                                rev        rare        0.094449  0.000000
                                           typical     0.087905  0.000000
CM: token property              fwd        rare        0.265423  0.343284
                                           typical     0.258333  0.277778
                                rev        rare        0.250000  0.029851
                                           typical     0.250000  0.129630
CP: category property           fwd        typical     0.248039  0.294118
                                rev        typical     0.159384  0.058824
CP: token property              fwd        rare        0.236161  0.291667
                                           typical     0.236161  0.281250
                                rev        rare        0.140402  0.083333
                                           typical     0.140402  0.104167

## Key Findings

- FT and ROME both successfully edit category memberships in forward queries (94%)
- FT and ROME both fail completely to edit for reverse queries (as expected)
- Category membership edits by ROME do *somewhat* generalize across category properties (greater than chance) (possibly above chance for FT as well)
- Edits generalize better for uncommon or atypical exemplars -- consistent with idea that it is easier to edit less popular knowledge (can cite that paper).
- IKE offers the best generalization performance
- (near) Ceiling performance for IKE on all reversed-queries is likely just an artifact of having the token string in the prompt--probably affects RippleEdit findings substantially as well.

## Conclusions

- We introduce a challenging benchmark for editing generalization quality
- ROME produces *some* generalization, giving some hope for localization and editing methods that modify LLM representations
- May offer hints to implicit structure of representations inside LLMs
